In [26]:
import boto3
import pandas as pd

# Neccessary for reading csv's off AWS Object Storage
from io import StringIO

# The timedelta module allows for easy selection of previous or later dates
from datetime import datetime as dt
from datetime import timedelta as td

In [27]:
# The date from which we want to pull all date going forward
arg_date = '2022-12-29'

In [28]:
# Input date needs to be arg_date-1 to populate prev_closing_price for arg_date, and needs to be DATETIME formate

# Convert string 'arg_date' into datetime
# datetime converts string into date and time, so the .date() func isolates date only
# timedelta lets you specify a timeframe to add/subtract from object

arg_date_dt = dt.strptime(arg_date, '%Y-%m-%d').date() - td(days=1)

In [29]:
arg_date_dt

datetime.date(2022, 12, 28)

In [2]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('xetra-1234')

bucket_obj1 = bucket.objects.filter(Prefix='2022-03-15')
bucket_obj2 = bucket.objects.filter(Prefix='2022-03-16')

objects = [obj for obj in bucket_obj1] + [obj for obj in bucket_obj2]

In [3]:
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-03-15/2022-03-15_BINS_XETR10.csv'),
 s3.Object

In [4]:
# Temp df to obtain column names
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')

In [5]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [6]:
# Func to read all csv files and concatenate into single df
df_all = pd.DataFrame(columns=df_init.columns)

for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_all = pd.concat(objs=[df_all, df], ignore_index=True)

In [7]:
# Remove unused columns from df_all
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,DE000A0DJ6J9,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421
1,DE000A0D6554,2022-03-15,08:00,16.58,16.58,16.43,16.45,21537
2,DE000A0D9PT0,2022-03-15,08:00,201.10,202.40,201.00,202.40,1585
3,DE000A0JL9W6,2022-03-15,08:00,62.55,63.20,62.55,63.20,1539
4,DE000A0LD6E6,2022-03-15,08:00,61.40,61.40,60.95,61.05,329
...,...,...,...,...,...,...,...,...
251797,US02005N1000,2022-03-16,20:30,37.80,37.80,37.80,37.80,0
251798,US2533931026,2022-03-16,20:30,101.00,101.00,101.00,101.00,0
251799,US89832Q1094,2022-03-16,20:30,52.50,52.50,52.50,52.50,0
251800,GB00BNYK8G86,2022-03-16,20:30,5.20,5.20,5.20,5.20,0


# Transformations

### Get opening price per ISIN and day

In [8]:
df_all['OpeningPrice'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

In [9]:
df_all[df_all['ISIN']=='DE000A0DJ6J9']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,OpeningPrice
0,DE000A0DJ6J9,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421,37.34
158,DE000A0DJ6J9,2022-03-15,08:01,37.28,37.28,37.20,37.20,2157,37.34
294,DE000A0DJ6J9,2022-03-15,08:02,37.14,37.28,37.06,37.06,2284,37.34
578,DE000A0DJ6J9,2022-03-15,08:03,37.06,37.06,36.92,36.94,1608,37.34
2999,DE000A0DJ6J9,2022-03-15,08:05,36.90,36.90,36.78,36.78,2449,37.34
...,...,...,...,...,...,...,...,...,...
246765,DE000A0DJ6J9,2022-03-16,16:25,38.14,38.14,38.10,38.10,430,38.38
247416,DE000A0DJ6J9,2022-03-16,16:27,38.14,38.14,38.08,38.08,193,38.38
247734,DE000A0DJ6J9,2022-03-16,16:28,38.10,38.10,38.06,38.10,316,38.38
248107,DE000A0DJ6J9,2022-03-16,16:29,38.08,38.18,38.08,38.12,374,38.38


### Get closing price per ISIN and day

In [10]:
df_all['ClosingPrice'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['EndPrice'].transform('last')

In [11]:
df_all[df_all['ISIN']=='DE000A0DJ6J9']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,OpeningPrice,ClosingPrice
0,DE000A0DJ6J9,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421,37.34,37.06
158,DE000A0DJ6J9,2022-03-15,08:01,37.28,37.28,37.20,37.20,2157,37.34,37.06
294,DE000A0DJ6J9,2022-03-15,08:02,37.14,37.28,37.06,37.06,2284,37.34,37.06
578,DE000A0DJ6J9,2022-03-15,08:03,37.06,37.06,36.92,36.94,1608,37.34,37.06
2999,DE000A0DJ6J9,2022-03-15,08:05,36.90,36.90,36.78,36.78,2449,37.34,37.06
...,...,...,...,...,...,...,...,...,...,...
246765,DE000A0DJ6J9,2022-03-16,16:25,38.14,38.14,38.10,38.10,430,38.38,38.16
247416,DE000A0DJ6J9,2022-03-16,16:27,38.14,38.14,38.08,38.08,193,38.38,38.16
247734,DE000A0DJ6J9,2022-03-16,16:28,38.10,38.10,38.06,38.10,316,38.38,38.16
248107,DE000A0DJ6J9,2022-03-16,16:29,38.08,38.18,38.08,38.12,374,38.38,38.16


### Aggregations

In [12]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('OpeningPrice', 'min'), 
                                                              closing_price_eur=('ClosingPrice', 'min'), 
                                                              minimum_price_eur=('MinPrice', 'min'),
                                                              maximum_price_eru=('MaxPrice','max'),
                                                              daily_traded_volume=('TradedVolume','sum'))

In [13]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eru,daily_traded_volume
0,AT000000STR1,2022-03-15,36.4000,37.0500,36.4000,37.5500,536
1,AT000000STR1,2022-03-16,37.5000,37.1500,37.1500,37.5500,816
2,AT00000FACC2,2022-03-15,7.8800,7.7100,7.6100,7.8800,68
3,AT00000FACC2,2022-03-16,7.9500,8.0800,7.9500,8.0800,696
4,AT0000606306,2022-03-15,13.3000,13.3600,12.8800,13.5700,24002
...,...,...,...,...,...,...,...
6387,XS2427474023,2022-03-16,24.8060,24.8060,24.8060,24.8060,0
6388,XS2434891219,2022-03-15,3.4840,3.5828,3.4840,3.5828,700
6389,XS2434891219,2022-03-16,3.6134,3.6558,3.6134,3.6844,27224
6390,XS2437455608,2022-03-15,27.0020,27.0020,27.0020,27.0020,0


### Percent change from previous closing price

In [14]:
df_all['previous_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [15]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eru,daily_traded_volume,previous_closing_price
0,AT000000STR1,2022-03-15,36.4000,37.0500,36.4000,37.5500,536,NaN
1,AT000000STR1,2022-03-16,37.5000,37.1500,37.1500,37.5500,816,37.0500
2,AT00000FACC2,2022-03-15,7.8800,7.7100,7.6100,7.8800,68,NaN
3,AT00000FACC2,2022-03-16,7.9500,8.0800,7.9500,8.0800,696,7.7100
4,AT0000606306,2022-03-15,13.3000,13.3600,12.8800,13.5700,24002,NaN
...,...,...,...,...,...,...,...,...
6387,XS2427474023,2022-03-16,24.8060,24.8060,24.8060,24.8060,0,25.0960
6388,XS2434891219,2022-03-15,3.4840,3.5828,3.4840,3.5828,700,NaN
6389,XS2434891219,2022-03-16,3.6134,3.6558,3.6134,3.6844,27224,3.5828
6390,XS2437455608,2022-03-15,27.0020,27.0020,27.0020,27.0020,0,NaN


In [16]:
df_all['%_change_closing_price'] = (df_all['closing_price_eur'] - df_all['previous_closing_price']) / df_all['previous_closing_price'] * 100

In [17]:
df_all.drop(columns=['previous_closing_price'], inplace=True)

In [18]:
df_all = df_all.round(decimals=2)

In [19]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eru,daily_traded_volume,%_change_closing_price
0,AT000000STR1,2022-03-15,36.40,37.05,36.40,37.55,536,NaN
1,AT000000STR1,2022-03-16,37.50,37.15,37.15,37.55,816,0.27
2,AT00000FACC2,2022-03-15,7.88,7.71,7.61,7.88,68,NaN
3,AT00000FACC2,2022-03-16,7.95,8.08,7.95,8.08,696,4.80
4,AT0000606306,2022-03-15,13.30,13.36,12.88,13.57,24002,NaN
...,...,...,...,...,...,...,...,...
6387,XS2427474023,2022-03-16,24.81,24.81,24.81,24.81,0,-1.16
6388,XS2434891219,2022-03-15,3.48,3.58,3.48,3.58,700,NaN
6389,XS2434891219,2022-03-16,3.61,3.66,3.61,3.68,27224,2.04
6390,XS2437455608,2022-03-15,27.00,27.00,27.00,27.00,0,NaN
